In [23]:
import json
import pandas as pd

In [ ]:
# https://cloud.google.com/compute/docs/general-purpose-machines#n2-high-cpu
file = './data/ep_ce_instances.json'
f = open(file, 'r')
data = json.load(f)['data']
machine_types = {
    "f1-micro": {
        'vCPUs': 1,
        "memory_gb": 0.6,
        "bandwidth_gbps": 1
    },
    "g1-small": {
        'vCPUs': 1,
        "memory_gb": 1.7,
        "bandwidth_gbps": 1
    },
    "n2-highcpu-2": {
        'vCPUs': 2,
        "memory_gb": 2,
        "bandwidth_gbps": 10
    },
    "e2-micro": {
        'vCPUs': 2,
        "memory_gb": 1,
        "bandwidth_gbps": 1
    },
    "e2-small": {
        'vCPUs': 2,
        "memory_gb": 2,
        "bandwidth_gbps": 1
    },
}
disk_type_enum = {
    'PERSISTENT': 0
}

In [ ]:
len(data)

In [ ]:
gcp_data = list(filter(lambda x: x.get('provider') == 'GCP', data))

In [ ]:
len(gcp_data)

In [ ]:
ep_gcp_map = {}
for i in range(0, len(gcp_data)):
    variant = gcp_data[i]['variant']
    if variant.get('id'):
        instance_type = variant["machineType"].split('/')[-1]
        ep_gcp_map[variant['id']] = {
            "id": variant['id'],
            "disk_type": list(map(lambda x: x.get('type'), variant['disks']))[0],
            "instance_type": instance_type,
            "region": variant.get("region"),
            "vCPUs": machine_types[instance_type]["vCPUs"],
            "memory_gb": machine_types[instance_type]["memory_gb"],
            "bandwidth_gbps": machine_types[instance_type]["bandwidth_gbps"]
        }

In [ ]:
ep_gcp_map

In [ ]:
gcp_ce_file = './data/gcp_cost.csv'
df_cost = pd.read_csv(gcp_ce_file, header=0, sep=',', dtype =  {"gcpId": str})

In [ ]:
df_cost

In [ ]:
# df['gcpId'] = df.apply(lambda row: str(ep_gcp_map.get(row.epId)), axis=1)
df_new = df_cost
def extract_variant(row, name):
    record = ep_gcp_map.get(row.gcpId)
    if not record:
        return ''
    return record.get(name)
df_new['instance_type'] = df_new.apply(lambda row: extract_variant(row, 'instance_type'), axis=1)
df_new['disk'] = df_new.apply(lambda row: extract_variant(row, 'disk_type'), axis=1)
df_new['vCPUs'] = df_new.apply(lambda row: extract_variant(row, 'vCPUs'), axis=1)
df_new['memory_gb'] = df_new.apply(lambda row: extract_variant(row, 'memory_gb'), axis=1)
df_new['bandwidth_gbps'] = df_new.apply(lambda row: extract_variant(row, 'bandwidth_gbps'), axis=1)
df_new['instanceId'] = df_new['gcpId']
df_new['datetime'] = df_new['date']

In [ ]:
df_filtered = df_new.loc[df_new['gcpId'] != None]

In [ ]:
df_filtered[['cost', 'datetime', 'costInUsd', 'instanceId', 'instance_type', 'disk', 'vCPUs', 'memory_gb', 'bandwidth_gbps']].to_csv('./data/gcp_cost_with_instance_info.csv')